In [1]:
import pandas as pd
import numpy as np
from keras.utils import plot_model
from tensorflow.keras import utils

In [ ]:
from keras.layers import (
    Input,
    Activation,
    Dense,
    Flatten,
    Dropout,
    GRU
)
from keras.layers import (
    Conv1D,
    Conv2D,
    MaxPooling1D,
    MaxPooling2D,
    AveragePooling1D
)
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import add
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam
import keras 
from keras import backend as K

In [ ]:
from tensorflow.keras import Model
from sklearn import preprocessing
full_scale = preprocessing.MinMaxScaler()
output_scale = preprocessing.MinMaxScaler()

In [ ]:
btc = pd.read_csv("../data/Bitstamp_BTCUSD_1h.csv", skiprows=1, index_col="date")
btc.index = pd.to_datetime(btc.index)
btc.sort_index(inplace=True)
btc["return"] = btc["close"] - btc["open"]  
btc["state"] = np.where(btc["return"] > 0, 1, 0)
btc["prev_vol_btc"] = btc["Volume BTC"].shift(1)
btc["prev_vol_USD"] = btc["Volume USD"].shift(1)
btc["prev_high"] = btc["high"].shift(1)
btc["prev_low"] = btc["low"].shift(1)
btc["pct_change"] = (btc["close"] - btc["open"])/ btc["open"] * 100 
btc = btc.iloc[1:]
train = btc[btc.index.year <= 2023]
train_y = output_scale.fit_transform(train[23:]["pct_change"].to_numpy().reshape(-1,1))

train_scaled = pd.DataFrame(full_scale.fit_transform(train.iloc[:,2:]), index=train.index, columns = train.columns[2:])
test = pd.DataFrame(full_scale.transform(btc.iloc[btc.index.year > 2023, 2:]), index=btc.iloc[btc.index.year > 2023].index, columns = btc.columns[2:])
"""
train = btc[btc.index.year <= 2023]
test = btc[btc.index.year > 2023]
train["open_scale"] = open_scale.fit_transform(train["open"].to_numpy().reshape(-1,1))
test["open_scale"] = open_scale.transform(test["open"].to_numpy().reshape(-1,1))
train["close_scale"] = open_scale.fit_transform(train["close"].to_numpy().reshape(-1,1))
test["close_scale"] = open_scale.transform(test["close"].to_numpy().reshape(-1,1))
train["prev_vol_btc_scale"] = vol_scale.fit_transform(train["prev_vol_btc"].to_numpy().reshape(-1,1))
test["prev_vol_btc_scale"] = vol_scale.transform(test["prev_vol_btc"].to_numpy().reshape(-1,1))
"""

In [ ]:
"""
btc["sma3"] = btc["open"].rolling(3).mean()
btc["sma6"] = btc["open"].rolling(6).mean()
btc["sma9"] = btc["open"].rolling(9).mean()
btc["sma12"] = btc["open"].rolling(12).mean()
btc["sma1d"] = btc["open"].rolling(24).mean()
btc["ewa3"] = btc["open"].ewm(span=3, min_periods=2).mean()
btc["ewa6"] = btc["open"].ewm(span=6, min_periods=5).mean()
btc["ewa9"] = btc["open"].ewm(span=9, min_periods=8).mean()
btc["ewa12"] = btc["open"].ewm(span=12, min_periods=11).mean()
btc["ewa1d"] = btc["close"].ewm(span=24, min_periods=23).mean"""

In [ ]:
train_x = np.hstack((np.lib.stride_tricks.sliding_window_view(train_scaled["open"], 24), 
                     np.lib.stride_tricks.sliding_window_view(train_scaled["prev_high"], 24),
                     np.lib.stride_tricks.sliding_window_view(train_scaled["prev_low"], 24),
                     np.lib.stride_tricks.sliding_window_view(train_scaled["prev_vol_btc"], 24),
                     np.lib.stride_tricks.sliding_window_view(train_scaled["prev_vol_USD"], 24)
                    )).reshape(-1,24,5)
test_x = np.hstack((np.lib.stride_tricks.sliding_window_view(test["open"], 24),
                    np.lib.stride_tricks.sliding_window_view(test["prev_high"], 24),
                    np.lib.stride_tricks.sliding_window_view(test["prev_low"], 24),
                    np.lib.stride_tricks.sliding_window_view(test["prev_vol_btc"], 24),
                     np.lib.stride_tricks.sliding_window_view(test["prev_vol_USD"], 24))
                  ).reshape(-1,24,5)
                           
test_y = output_scale.transform(btc.iloc[btc.index.year > 2023].iloc[23:]["pct_change"].to_numpy().reshape(-1,1))

#train_y= min_max_scaler.fit_transform(train.iloc[23:]["return"].to_numpy().reshape(-1,1))
#test_y = min_max_scaler.transform(test.iloc[23:]["return"].to_numpy().reshape(-1,1))



In [ ]:
pd.DataFrame(test_y).describe()

In [ ]:
model = keras.Sequential(name="model_conv1D")
model.add(keras.layers.Input(shape=(24,5)))
model.add(Conv1D(32, 5, activation='relu'))
model.add(Dropout(0.1))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Dropout(0.1))
model.add(GRU(units = 128, return_sequences=True))
model.add(Dropout(0.5))
model.add(GRU(units = 128, return_sequences=True))
model.add(Dropout(0.5))
model.add(GRU(units = 128))
model.add(Dropout(0.5))
model.add(Dense(units = 64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='linear'))


optimizer = keras.optimizers.Adam()
model.compile(loss='mae',optimizer=optimizer,metrics=['mse', 'root_mean_squared_error'])

In [ ]:
model.summary()

In [ ]:
monitor = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
model.fit(train_x, train_y, batch_size=64, validation_data=(test_x,test_y), callbacks=[monitor], epochs=100)

In [ ]:
y = output_scale.inverse_transform(model.predict(test_x))

In [ ]:
test["u_close"] = output_scale.inverse_transform(test["close"].to_numpy().reshape(-1,1)) 

In [ ]:
test["u_close"]

In [ ]:
test["y_pred"] = np.append(np.repeat(np.nan, 23), y) * test["open"]

In [ ]:
test["y_pred"]

In [ ]:
test.[["u_close", "y_pred"]].plot()

In [ ]:
pd.to_datetime("2024-03-15 18:00:00")

In [ ]:
test["d_close"] = test["close"].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
test["d_pred"] = test["y_pred"].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
test["lazy"] = test["open"].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])

In [ ]:
from sklearn.metrics import root_mean_squared_error
ok = test.dropna(axis=0)

In [ ]:
root_mean_squared_error(ok["d_close"], ok["d_pred"])

In [ ]:
root_mean_squared_error(ok["d_close"], ok["lazy"])

In [ ]:
test["delta"] = (test["d_close"] - test["d_pred"])**
test["delta_lazy"] = (test["d_close"] - test["d_pred"])**2

In [ ]:
test["delta"].plot()

In [ ]:
btc["return_scaled"].describe()

In [ ]:
inputs = Input(shape=(24,1), name='input')
x = Conv1D(32, 3, activation="relu", padding="same", name="convLayer")(inputs)
x = Dropout(0.1)(x)

x = Conv1D(64, 3, activation="relu")(x)
x = Dropout(0.1)(x)

x = GRU(128, activation="relu", return_sequences=True)(x)
x = Dropout(0.5)(x)


x = GRU(128, activation="relu")(x)
x = Dropout(0.5)(x)

x = Dense(64, activation="relu")(x)
x = Dropout(0.5)(x)

x = Dense(32, activation="relu")(x)
x = Dropout(0.5)(x)

output1 = Dense(1, name='cont_out')(x)
output2 = Dense(3, activation='softmax', name='cat_out')(x)

model = Model(inputs=inputs, outputs=[output1, output2])



In [ ]:
np_utils.to_categorical(btc["state"], 3)

In [ ]:
btc["state"].value_counts()

In [ ]:
btc.iloc[23:][["state", "return"]].to_numpy()

In [ ]:
y = btc.iloc[:24]

In [ ]:
btc.iloc[:25][["open","close"]]

In [ ]:
btc.index = pd.to_datetime(btc.index)
btc.sort_index(inplace=True)

In [ ]:
btc["diff"] = btc["close"] - btc["open"]

In [ ]:
btc["diff"].idxmin()

In [ ]:
btc["sma3"] = btc["open"].rolling(3).mean()
btc["sma6"] = btc["open"].rolling(6).mean()
btc["sma9"] = btc["open"].rolling(9).mean()
btc["sma12"] = btc["open"].rolling(12).mean()
btc["sma1d"] = btc["open"].rolling(24).mean()

In [ ]:
btc["ewa3"] = btc["open"].ewm(span=3, min_periods=2).mean()
btc["ewa6"] = btc["open"].ewm(span=6, min_periods=5).mean()
btc["ewa9"] = btc["open"].ewm(span=9, min_periods=8).mean()
btc["ewa12"] = btc["open"].ewm(span=12, min_periods=11).mean()
btc["ewa1d"] = btc["close"].ewm(span=24, min_periods=23).mean()

In [ ]:
btc['12hrSTD'] = btc['open'].rolling(window=12).std()
btc['Upper'] = btc['sma12'] + (btc['12hrSTD'] * 2)
btc['Lower'] = btc['sma12'] - (btc['12hrSTD'] * 2)

In [ ]:
btc["return"] = btc["diff"] - (btc["close"] * .0003)

In [ ]:
btc["state"] = np.where(btc["return"] > 0, 0,1)
#btc["state"] = np.where((btc["return"] < 10) & (btc["return"] > -10), 1, btc["state"])

In [ ]:
btc["state"].value_counts()

In [ ]:
print(model.summary())

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss={'y1_output': 'mse', 'y2_output': 'mse'},
              metrics={'y1_output': tf.keras.metrics.RootMeanSquaredError(),
                       'y2_output': tf.keras.metrics.RootMeanSquaredError()})

In [ ]:
model = Sequential()
model.add(Input(shape=(10,1)))
model.add(Conv1D(32, 3, activation='relu'))
model.add(Dropout(0.1))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Dropout(0.1))
model.add(GRU(units = 128, return_sequences=True))
model.add(Dropout(0.5))
model.add(GRU(units = 128))
model.add(Dropout(0.5))
model.add(Dense(units = 64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=2, activation='linear'))

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.MeanSquaredError(
    reduction="sum_over_batch_size", name="mean_squared_error"),
    # List of metrics to monitor
    metrics=[keras.metrics.MeanSquaredError()],
)


In [ ]:
train = btc[btc.index.year < 2024]
test = btc[btc.index.year >= 2024]

In [ ]:
train_x = train[["open", "sma3", "sma6", "sma9", "sma12", "sma1d", "ewa3", "12hrSTD", "Upper", "Lower"]]
train_y = train[["state", "diff"]]
test_x = test[["open", "sma3", "sma6", "sma9", "sma12", "sma1d", "ewa3", "12hrSTD", "Upper", "Lower"]]
test_y = test[["state", "diff"]]


In [ ]:
model.fit(train_x, train_y["diff"],
         batch_size=64,
    epochs=5,
)

In [ ]:
model.predict(test_x)